# Predicting Football Match Results

This notebook builds a machine learning model that can be used to predict professional soccer games. 

The model uses Logistic Regression, built from the given data from all the matches. 

All the exlanation and wall-through steps are included in the ipython notebook called football_prediction.jpynd. 
There are two python files that are used by this notebook:

* features: Turns raw statistics into features that get fed into the machine learning model. These features combine information from team_attribute and player_attributes for each of the matchs. 
* models: Helper methods for cleaning the data and building and running the logistic regression model.

Data Description: 
+ match: thông tin trận đấu giữa 2 đội home_team và away_team
    - date: ngày diễn ra trận đấu
    - home/away_team_api_id: id đội nhà/đội khách
    - home/away_player_(1..11): id cầu thủ đội nhà/đội khách
    - home/away_player_X/Y(1..11): đội hình ra sân của đội nhà/đội khách (tọa độ X, Y)
    - goal, shoton, ..., corner, possession: diễn tiến trận đấu 
    - B365H, B365D, ..., BSD, BSA: tỉ lệ đặt cược
+ team, player: thông tin đội bóng, cầu thủ
+ team_attributes: thông số đội bóng
    - team_api_id: id đội bóng
    - date: ngày lấy thông số
    - buildUpPlaySpeed, ..., defenceDefenderLineClass: thông số đội bóng
+ player_attributest: thông tin, thông số cầu thủ
    - player_api_id: id cầu thủ
    - date: ngày lấy thông số
    - overall_rating, ..., gk_reflexes: thông số cầu thủ
+ country, league: thông tin quốc gia và giải đấu

Tasks:
+ Analyse and explain whether the following factor impacts the final result of the match:
    + Team Profile
    + Players Profile
    + Starting Players from both teams
    + Historical matches between the two teams
    + Other factors
+ Build a predictive model using the above analysis to predict the outcome of the game (which team is more likely to win)
+ Choose the appropriate evaluation metrics for the task

In [1]:
# load data
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
match = pd.read_json("data/match")
team = pd.read_json("data/team")
player = pd.read_json("data/player")
country = pd.read_json("data/country")
league = pd.read_json("data/league")
player_attributes = pd.read_json("data/player_attributes")
team_attributes = pd.read_json("data/team_attributes")

## Data exploration

### match

In [185]:
match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27000 entries, 0 to 26999
Columns: 115 entries, id to BSA
dtypes: datetime64[ns](1), float64(96), int64(9), object(9)
memory usage: 23.7+ MB


In [256]:
match.shape

(27000, 115)

In [186]:
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,2.5,2.50,3.2,2.75,2.45,3.25,2.6,2.50,3.25,2.62
1,25768,24558,24558,2014/2015,36,2015-05-29,1775174,10199,10179,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16288,15722,15722,2010/2011,19,2011-03-18,838672,8322,8021,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14256,13274,13274,2011/2012,16,2011-12-10,1028625,8611,8464,2,...,17.0,1.18,7.0,19.00,1.17,6.00,15.0,1.17,6.00,17.00
4,20964,19694,19694,2013/2014,29,2014-03-15,1474488,8426,10251,3,...,2.8,2.55,3.3,2.90,NaN,NaN,NaN,NaN,NaN,NaN


### team

In [187]:
team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


### player

In [188]:
player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11060 non-null  int64  
 1   player_api_id       11060 non-null  int64  
 2   player_name         11060 non-null  object 
 3   player_fifa_api_id  11060 non-null  int64  
 4   birthday            11060 non-null  object 
 5   height              11060 non-null  float64
 6   weight              11060 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 605.0+ KB


### country

In [189]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11 non-null     int64 
 1   name    11 non-null     object
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


### league

In [190]:
league.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11 non-null     int64 
 1   country_id  11 non-null     int64 
 2   name        11 non-null     object
dtypes: int64(2), object(1)
memory usage: 392.0+ bytes


### player_attributes

In [191]:
player_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   183978 non-null  int64         
 1   player_fifa_api_id   183978 non-null  int64         
 2   player_api_id        183978 non-null  int64         
 3   date                 183978 non-null  datetime64[ns]
 4   overall_rating       183142 non-null  float64       
 5   potential            183142 non-null  float64       
 6   preferred_foot       183142 non-null  object        
 7   attacking_work_rate  180748 non-null  object        
 8   defensive_work_rate  183142 non-null  object        
 9   crossing             183142 non-null  float64       
 10  finishing            183142 non-null  float64       
 11  heading_accuracy     183142 non-null  float64       
 12  short_passing        183142 non-null  float64       
 13  volleys       

### team_attributes

In [192]:
team_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              1458 non-null   int64         
 1   team_fifa_api_id                1458 non-null   int64         
 2   team_api_id                     1458 non-null   int64         
 3   date                            1458 non-null   datetime64[ns]
 4   buildUpPlaySpeed                1458 non-null   int64         
 5   buildUpPlaySpeedClass           1458 non-null   object        
 6   buildUpPlayDribbling            489 non-null    float64       
 7   buildUpPlayDribblingClass       1458 non-null   object        
 8   buildUpPlayPassing              1458 non-null   int64         
 9   buildUpPlayPassingClass         1458 non-null   object        
 10  buildUpPlayPositioningClass     1458 non-null   object        
 11  chan

## Features Engineering

### Building features
This will return a pandas dataframe that contains the features that will be used to build a model. <br>
The following steps will be taken:
1. data cleaning: converting data into appropriate format)
2. create target columns: name ***points**, which indicate the outcome of the game. 3 points for a win, 1 point for a draw, 0 for a loss
3. create features: create new columns to exacts information from raw data that can be fed into the model

### data cleaning

In [3]:
def data_cleaning():
    # clean date values
    match['date'] = pd.to_datetime(match['date'], format='%Y-%m-%d')
    player_attributes['date'] = pd.to_datetime(player_attributes['date'], format='%Y-%m-%d')
    team_attributes['date'] = pd.to_datetime(team_attributes['date'], format='%Y-%m-%d')
    # remove duplicates
    match.drop_duplicates(inplace=True)
    team.drop_duplicates(inplace=True)
    player.drop_duplicates(inplace=True)
    country.drop_duplicates(inplace=True)
    league.drop_duplicates(inplace=True)
    player_attributes.drop_duplicates(inplace=True)
    team_attributes.drop_duplicates(inplace=True)

In [4]:
data_cleaning()

### create target columns

The following columns are target variables that we will be attempting to predict. These columns must be dropped before any prediction is done, but are useful when building a model. The models that we will build below will just try to predict outcome (points)

* points: The outcome of the game. 3 points for a win, 1 point for a draw, 0 for a loss. (Points are not goals!)

In [5]:
match_prep = match.copy()

In [6]:
def compute_points(df):
    if df.home_team_goal > df.away_team_goal:
        return 3
    elif df.home_team_goal == df.away_team_goal:
        return 0
    else:
        return 1
match_prep['points'] = match_prep.apply(compute_points, axis=1)

In [7]:
match_prep.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,points
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,2.50,3.2,2.75,2.45,3.25,2.6,2.50,3.25,2.62,1
1,25768,24558,24558,2014/2015,36,2015-05-29,1775174,10199,10179,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,16288,15722,15722,2010/2011,19,2011-03-18,838672,8322,8021,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,14256,13274,13274,2011/2012,16,2011-12-10,1028625,8611,8464,2,...,1.18,7.0,19.00,1.17,6.00,15.0,1.17,6.00,17.00,3
4,20964,19694,19694,2013/2014,29,2014-03-15,1474488,8426,10251,3,...,2.55,3.3,2.90,NaN,NaN,NaN,NaN,NaN,NaN,3


### Team Profile 

In [8]:
team_attributes.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [8]:
team_attributes_feats = list(set(team_attributes.columns) - set(['id', 'team_fifa_api_id']))

In [280]:
def get_team_attributes(match, team_attributes):
    '''
    for each team in match data, find all the corresponding data about the team from the team_attributes set
    the team_attributes taken is the one which date is the closest date to the date of the match
    '''
    
    def _get_team_attributes(match, team_attributes, teamid='home_team_api_id'):
        team_api_ids = list(match[teamid].unique())

        for count, team in enumerate(team_api_ids):
            match_info = match[(match[teamid]==team)]
            team_attri_info = team_attributes[team_attributes['team_api_id']==team]
            match_dates = list(match_info['date'].unique())
            for count_, date in enumerate(match_dates): 
                # get the team_attributes of the closest date
                team_attri_closest = team_attri_info[(team_attri_info['date']<= date)]
                team_attri_closest.sort_values(by='date', ascending=False, inplace=True)
                team_attri_closest = team_attri_closest.head(1)
                # drop unused cols
                team_attri_closest.drop(['id', 'team_fifa_api_id', 'date'], axis=1, inplace=True)
                # join team_attri_closest to the match_info set
                __match_joined = pd.merge(match_info[(match_info.date == date)], team_attri_closest, how='left', left_on=[teamid], right_on=['team_api_id'])
                # concat the joined sets by date
                if count_ == 0:
                    _match_joined = __match_joined
                else:
                    _match_joined = pd.concat([_match_joined, __match_joined], ignore_index=True)
#                 print(f'count: {count_}, date: {date},  no.matchs: {_match_joined.shape}')

            # concat the joined sets by team
            if count == 0:
                match_joined = _match_joined
            else:
                match_joined = pd.concat([match_joined, _match_joined], ignore_index=True)
#             print(f'count: {count}, no.joined matchs: {match_joined.shape}')
                
        return match_joined
    
    # get the team attributes for home_team and away_team separately
    match_home_team_attri = _get_team_attributes(match, team_attributes, teamid='home_team_api_id')
    match_away_team_attri = _get_team_attributes(match, team_attributes, teamid='away_team_api_id')
    
    # rename the features with prefix h_, aw to indicate feature for home_team, away_team
    feats = list(set(team_attributes.columns) - set(['id', 'team_fifa_api_id', 'team_api_id', 'date']))
    for feat in feats:
        match_home_team_attri.rename(columns={feat: f'home_{feat}'}, inplace=True)
        match_away_team_attri.rename(columns={feat: f'away_{feat}'}, inplace=True)
    
    # join the 2 dataframes
    join_cols = ['id', 'country_id', 'league_id', 'season', 'stage','date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']
    match_home_team_attri = match_home_team_attri[join_cols+[f'home_{feat}' for feat in feats]]
    match_away_team_attri = match_away_team_attri[join_cols+[f'away_{feat}' for feat in feats]]
    match_team_attri = pd.merge(match_home_team_attri, match_away_team_attri, how='inner', on=join_cols)
    
    # create compared features indicate the difference between home_team and away_team
    numerical_feats = team_attributes.drop(['id', 'team_fifa_api_id', 'team_api_id', 'date'], axis=1).select_dtypes(include='int64').columns.to_list()
    categorical_feats = team_attributes.select_dtypes(include='object').columns.to_list()
    
    for feat in feats:
        if feat in numerical_feats:
            match_team_attri[f'isHigher_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'home_{feat}'] > x[f'away_{feat}'] else 0, axis=1)
        if feat in categorical_feats:
            match_team_attri[f'isDiff_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'home_{feat}'] != x[f'away_{feat}'] else 0, axis=1)
    
#     # return only compared features 
#     compared_numerical_feats = [f'isHigher_{feat}' for feat in numerical_feats]
#     compared_categorical_feats = [f'isDiff_{feat}' for feat in categorical_feats]
#     match_info = ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal','away_team_goal']
    
    return match_team_attri

In [281]:
# compute features for the sample of the first 100 matchs
match_team_attri = get_team_attributes(match.head(100), team_attributes)

In [279]:
match_team_attri

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,isDiff_chanceCreationPassingClass,isDiff_defencePressureClass,isHigher_chanceCreationShooting,isDiff_buildUpPlayPositioningClass,isHigher_defenceTeamWidth,isHigher_defenceAggression,isDiff_buildUpPlayDribblingClass,isDiff_buildUpPlaySpeedClass,isDiff_chanceCreationCrossingClass,isDiff_chanceCreationPositioningClass
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,0,1,1,0,1,0,0,0,1,0
1,25768,24558,24558,2014/2015,36,2015-05-29,1775174,10199,10179,3,...,0,0,1,0,0,1,0,0,0,0
2,16288,15722,15722,2010/2011,19,2011-03-18,838672,8322,8021,1,...,1,0,1,0,0,0,0,0,0,0
3,14256,13274,13274,2011/2012,16,2011-12-10,1028625,8611,8464,2,...,0,1,1,0,1,1,0,0,1,1
4,20964,19694,19694,2013/2014,29,2014-03-15,1474488,8426,10251,3,...,1,1,0,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7813,7809,7809,2008/2009,1,2008-08-17,499321,9810,8177,0,...,1,1,0,1,0,0,1,1,1,1
96,14445,13274,13274,2011/2012,4,2011-08-27,1028512,10219,9761,2,...,0,0,0,1,1,0,0,0,0,0
97,20165,19694,19694,2010/2011,11,2010-11-06,840292,8066,9927,1,...,1,1,0,1,0,0,1,1,1,1
98,891,1,1,2011/2012,24,2012-02-04,1032883,9989,9985,0,...,0,0,0,0,0,1,0,1,0,1


In [9]:
# match_team_attri.to_csv('data/match_team_attri_1000.csv', index=False)

# load saved file
match_team_attri = pd.read_csv('data/feat_team_attributes.csv')

In [10]:
match_team_attri.drop(['home_buildUpPlayDribbling', 'away_buildUpPlayDribbling'], axis=1, inplace=True)
match_team_attri.dropna(inplace=True)

In [11]:
match_team_attri

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,away_chanceCreationPositioningClass,away_chanceCreationCrossingClass,away_defenceTeamWidthClass,away_buildUpPlaySpeedClass,away_defenceTeamWidth,away_defencePressureClass,away_defenceAggressionClass,away_buildUpPlayPassingClass,away_chanceCreationPassing,away_chanceCreationPassingClass
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,Organised,Normal,Narrow,Fast,30.0,Deep,Double,Mixed,50.0,Normal
8,20100,19694,19694,2009/2010,36,2010-05-01,820496,8596,9800,1,...,Organised,Lots,Wide,Fast,70.0,Medium,Double,Long,70.0,Risky
16,20105,19694,19694,2009/2010,37,2010-05-05,820502,8596,8467,0,...,Organised,Normal,Narrow,Fast,30.0,Deep,Double,Mixed,50.0,Normal
23,20080,19694,19694,2009/2010,33,2010-04-10,659058,8596,9800,2,...,Organised,Lots,Wide,Fast,70.0,Medium,Double,Long,70.0,Risky
29,20046,19694,19694,2009/2010,28,2010-03-07,659030,8596,9925,0,...,Organised,Lots,Wide,Fast,70.0,Medium,Double,Long,65.0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25952,5431,4769,4769,2009/2010,35,2010-05-02,654728,4170,9831,3,...,Organised,Little,Wide,Fast,70.0,High,Double,Long,60.0,Normal
25953,5361,4769,4769,2009/2010,29,2010-03-20,654491,4170,8689,2,...,Organised,Normal,Wide,Balanced,70.0,High,Press,Long,30.0,Safe
25955,5411,4769,4769,2009/2010,33,2010-04-17,654708,4170,8592,1,...,Free Form,Normal,Normal,Balanced,65.0,Medium,Double,Mixed,70.0,Risky
25956,5341,4769,4769,2009/2010,27,2010-03-06,654445,4170,9748,0,...,Organised,Normal,Wide,Balanced,70.0,High,Press,Mixed,70.0,Risky


In [12]:
# create features
feats = list(set(team_attributes.columns) - set(['id', 'team_fifa_api_id', 'team_api_id', 'date']))
# create compared features indicate the difference between home_team and away_team
numerical_feats = team_attributes.drop(['id', 'team_fifa_api_id', 'team_api_id', 'date'], axis=1).select_dtypes(include='int64').columns.to_list()
categorical_feats = team_attributes.select_dtypes(include='object').columns.to_list()

for feat in feats:
    if feat in numerical_feats:
        match_team_attri[f'isHigher_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'home_{feat}'] > x[f'away_{feat}'] else 0, axis=1)
    if feat in categorical_feats:
        match_team_attri[f'isDiff_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'home_{feat}'] != x[f'away_{feat}'] else 0, axis=1)

In [13]:
# return only compared features 
compared_numerical_feats = [f'isHigher_{feat}' for feat in numerical_feats]
compared_categorical_feats = [f'isDiff_{feat}' for feat in categorical_feats]
match_info = ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal','away_team_goal']
match_team_attri = match_team_attri[match_info+compared_numerical_feats+compared_categorical_feats]

In [14]:
# create target column
def compute_points(df):
    if df.home_team_goal > df.away_team_goal:
        return 3
    elif df.home_team_goal == df.away_team_goal:
        return 0
    else:
        return 1
match_team_attri['points'] = match_team_attri.apply(compute_points, axis=1)
match_team_attri

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,isDiff_buildUpPlayPositioningClass,isDiff_chanceCreationPassingClass,isDiff_chanceCreationCrossingClass,isDiff_chanceCreationShootingClass,isDiff_chanceCreationPositioningClass,isDiff_defencePressureClass,isDiff_defenceAggressionClass,isDiff_defenceTeamWidthClass,isDiff_defenceDefenderLineClass,points
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,0,0,1,1,0,1,0,1,0,1
8,20100,19694,19694,2009/2010,36,2010-05-01,820496,8596,9800,1,...,0,1,0,0,0,0,0,0,0,0
16,20105,19694,19694,2009/2010,37,2010-05-05,820502,8596,8467,0,...,0,0,1,1,0,1,0,1,0,0
23,20080,19694,19694,2009/2010,33,2010-04-10,659058,8596,9800,2,...,0,1,0,0,0,0,0,0,0,3
29,20046,19694,19694,2009/2010,28,2010-03-07,659030,8596,9925,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25952,5431,4769,4769,2009/2010,35,2010-05-02,654728,4170,9831,3,...,0,0,1,0,0,0,1,0,1,0
25953,5361,4769,4769,2009/2010,29,2010-03-20,654491,4170,8689,2,...,0,1,0,0,0,0,0,0,0,3
25955,5411,4769,4769,2009/2010,33,2010-04-17,654708,4170,8592,1,...,0,1,0,1,1,1,1,1,0,1
25956,5341,4769,4769,2009/2010,27,2010-03-06,654445,4170,9748,0,...,0,1,0,1,0,0,0,0,0,0


In [16]:
match_team_attri.to_csv('data/match_team_attri.csv', index=False)

### train model

In [24]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import statsmodels.api as sm

In [17]:
import models
# reload(models)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Don't train on games that ended in a draw, since they have less signal.
train = match_team_attri.loc[match_team_attri['points'] != 1] 
# train = club_data

ignore_cols = ['id', 'country_id', 'league_id', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']
(model, test) = models.train_model(match_team_attri, ignore_cols)
print("\nRsquared: %0.03g" % model.prsquared)

KeyError: 'teamid'

In [343]:
def split(data, test_proportion=0.4):
    """ Splits a dataframe into a training set and a test set.
        Must be careful because back-to-back rows are expeted to
        represent the same game, so they both must go in the 
        test set or both in the training set.
    """
    
    train_vec = []
    if len(data) % 2 != 0:
        raise Exception('Unexpected data length')
    while len(train_vec) < len(data):
        rnd = random.random()
        train_vec.append(rnd > test_proportion) 
        train_vec.append(rnd > test_proportion)
            
    test_vec = [not val for val in train_vec]
    train = data[train_vec]
    test = data[test_vec]
    if len(train) % 2 != 0:
        raise Exception('Unexpected train length')
    if len(test) % 2 != 0:
        raise Exception('Unexpected test length')
    return (train, test)


def _extract_target(data, target_col):
    """ Removes the target column from a data frame, returns the target
        col and a new data frame minus the target. """
    target = data[target_col]
    train_df = data.copy()
    del train_df[target_col]
    return target, train_df


In [345]:
L1_ALPHA = 16.0
def build_model_logistic(target, data, acc=0.00000001, alpha=L1_ALPHA):
    """ Trains a logistic regresion model. target is the target.
        data is a dataframe of samples for training. The length of 
        target must match the number of rows in data.
    """ 
    data = data.copy()
    data['intercept'] = 1.0
    logit = sm.Logit(target, data, disp=False)
    return logit.fit_regularized(maxiter=1024, alpha=alpha, acc=acc, disp=False)


def validate(label, target, predictions, baseline=0.5, compute_auc=False,
             quiet=True):
    """ Validates binary predictions, computes confusion matrix and AUC.

      Given a vector of predictions and actual values, scores how well we
      did on a prediction. 

      Args:
        label: label of what we're validating
        target: vector of actual results
        predictions: predicted results. May be a probability vector,
          in which case we'll sort it and take the most confident values
          where baseline is the proportion that we want to take as True
          predictions. If a prediction is 1.0 or 0.0, however, we'll take
          it to be a true or false prediction, respectively.
        compute_auc: If true, will compute the AUC for the predictions. 
          If this is true, predictions must be a probability vector.
    """

    if len(target) != len(predictions):
        raise Exception('Length mismatch %d vs %d' % (len(target), 
                                                      len(predictions)))
    if baseline > 1.0:
        # Baseline number is expected count, not proportion. Get the proportion.
        baseline = baseline * 1.0 / len(target)

    zipped = sorted(zip(target, predictions), key=lambda tup: -tup[1])
    expect = len(target) * baseline
    
    (true_pos, true_neg, false_pos, false_neg) = (0, 0, 0, 0)
    for index in xrange(len(target)):
        (yval, prob) = zipped[index]
        if float(prob) == 0.0:
            predicted = False
        elif float(prob) == 1.0:
            predicted = True
        else:
            predicted = index < expect
        if predicted:
            if yval:
                true_pos += 1
            else:
                false_pos += 1 
        else:
            if yval:
                false_neg += 1
            else:
                true_neg += 1
    pos = true_pos + false_neg
    neg = true_neg + false_pos
    # P(1 | predicted(1)) and P(0 | predicted(f))
    pred_t = true_pos + false_pos
    pred_f = true_neg + false_neg
    prob1_t = true_pos * 1.0 / pred_t if pred_t > 0.0 else -1.0
    prob0_f = true_neg * 1.0 / pred_f if pred_f > 0.0 else -1.0
              
    # Lift = P(1 | t) / P(1)
    prob_1 = pos * 1.0 / (pos + neg)
    lift = prob1_t / prob_1 if prob_1 > 0 else 0.0
              
    accuracy = (true_pos + true_neg) * 1.0 / len(target)
              
    if compute_auc:
        y_bool =  [True if yval else False for (yval, _) in zipped]
        x_vec = [xval for (_, xval) in zipped]
        auc_value = roc_auc_score(y_bool, x_vec)
        fpr, tpr, _ = roc_curve(y_bool, x_vec)
        pl.plot(fpr, tpr, lw=1.5,
            label='ROC %s (area = %0.2f)' % (label, auc_value))
        pl.xlabel('False Positive Rate', fontsize=18)
        pl.ylabel('True Positive Rate', fontsize=18)
        pl.title('ROC curve', fontsize=18)
        auc_value = '%0.03g' % auc_value
    else:
        auc_value = 'NA'

    print('(%s) Lift: %0.03g Auc: %s' % (label, lift, auc_value))
    if not quiet:
        print('    Base: %0.03g Acc: %0.03g P(1|t): %0.03g P(0|f): %0.03g' % (
            baseline, accuracy, prob1_t, prob0_f))
        print('    Fp/Fn/Tp/Tn p/n/c: %d/%d/%d/%d %d/%d/%d' % (
            false_pos, false_neg, true_pos, true_neg, pos, neg, len(target)))

### Player Profile

In [242]:
player_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [ ]:
def get_player_attribute(match_df):
    # extract features for home_team
    home_player_cols = [f'home_player_{i}' for i in range(1,12)]
    home_info = match_df[['id', 'match_api_id', 'home_team_api_id','date']+ home_player_cols]
    home_player_ids = list(home_info[home_player_cols].iloc[0].values)
    match_date = home_info.date.iloc[0]
    # create a dataframe with all home_player info
    home_player_attribute = player_attributes[(player_attributes.date <= match_date) & (player_attributes.player_api_id.isin(home_player_ids))]
    home_player_attribute.sort_values(by='date', ascending=False, inplace=True)
    home_player_attribute.drop_duplicates(subset=['player_api_id'], keep='first', inplace=True)
    

In [282]:
s1 = match.head(1)

In [283]:
s1

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,19982,19694,19694,2009/2010,19,2010-03-13,658907,8596,8467,1,...,2.5,2.5,3.2,2.75,2.45,3.25,2.6,2.5,3.25,2.62


In [284]:
home_player_cols = [f'home_player_{i}' for i in range(1,12)]
home_player_cols

['home_player_1',
 'home_player_2',
 'home_player_3',
 'home_player_4',
 'home_player_5',
 'home_player_6',
 'home_player_7',
 'home_player_8',
 'home_player_9',
 'home_player_10',
 'home_player_11']

In [287]:
s1 = s1[['id', 'match_api_id', 'home_team_api_id','date']+ home_player_cols]

In [288]:
s1

,id,match_api_id,home_team_api_id,date,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11
0,19982,658907,8596,2010-03-13,43230.0,32553.0,35500.0,29049.0,32556.0,34662.0,35524.0,23254.0,95335.0,33060.0,32546.0


In [298]:
home_player_ids = list(s1[home_player_cols].iloc[0].values)
home_player_ids

[43230.0,
 32553.0,
 35500.0,
 29049.0,
 32556.0,
 34662.0,
 35524.0,
 23254.0,
 95335.0,
 33060.0,
 32546.0]

In [303]:
m_date = s1.date.iloc[0]

In [309]:
home_player_attribute = player_attributes[(player_attributes.date <= m_date) & (player_attributes.player_api_id.isin(home_player_ids))]
home_player_attribute.sort_values(by='date', ascending=False, inplace=True)
home_player_attribute.drop_duplicates(subset=['player_api_id'], keep='first', inplace=True)

In [310]:
home_player_attribute

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
23456,23457,164771,35500,2010-02-22,63.0,65.0,right,medium,medium,61.0,...,43.0,21.0,63.0,63.0,62.0,9.0,14.0,15.0,6.0,8.0
31699,31700,4240,35524,2010-02-22,63.0,70.0,right,medium,medium,45.0,...,56.0,62.0,56.0,63.0,41.0,9.0,21.0,48.0,21.0,21.0
155431,155432,186126,95335,2010-02-22,64.0,68.0,right,medium,medium,68.0,...,66.0,52.0,41.0,31.0,28.0,6.0,15.0,13.0,15.0,5.0
170965,170966,173987,29049,2010-02-22,61.0,63.0,left,low,high,58.0,...,54.0,8.0,65.0,67.0,61.0,14.0,15.0,8.0,14.0,14.0
150654,150655,178485,43230,2010-02-22,60.0,63.0,right,medium,medium,13.0,...,35.0,17.0,11.0,10.0,10.0,61.0,54.0,44.0,63.0,62.0
141647,141648,161973,32546,2010-02-22,61.0,65.0,right,None,_0,54.0,...,NaN,54.0,21.0,21.0,NaN,7.0,21.0,47.0,21.0,21.0
114834,114835,170630,32556,2010-02-22,57.0,62.0,left,medium,high,61.0,...,32.0,36.0,57.0,61.0,55.0,6.0,14.0,12.0,15.0,7.0
38282,38283,161651,32553,2010-02-22,65.0,78.0,right,low,medium,53.0,...,56.0,70.0,66.0,67.0,62.0,9.0,21.0,61.0,21.0,21.0
50537,50538,157589,33060,2010-02-22,62.0,68.0,left,medium,medium,46.0,...,59.0,61.0,43.0,39.0,37.0,12.0,20.0,24.0,20.0,20.0
24653,24654,139550,23254,2010-02-22,63.0,67.0,left,None,_0,61.0,...,NaN,65.0,37.0,56.0,NaN,13.0,23.0,58.0,23.0,23.0


In [318]:
# create features by summary statistics across all the players
numerical_features = home_player_attribute.select_dtypes(include='float').columns.to_list()
catergorical_features = home_player_attribute.select_dtypes(include='object').columns.to_list()
catergorical_features

['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']

In [320]:
for feat in numerical_features:
    s1[f'avg_{feat}_home_players'] = home_player_attribute[feat].mean()
s1

,id,match_api_id,home_team_api_id,date,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,...,avg_vision_home_players,avg_penalties_home_players,avg_marking_home_players,avg_standing_tackle_home_players,avg_sliding_tackle_home_players,avg_gk_diving_home_players,avg_gk_handling_home_players,avg_gk_kicking_home_players,avg_gk_positioning_home_players,avg_gk_reflexes_home_players
0,19982,658907,8596,2010-03-13,43230.0,32553.0,35500.0,29049.0,32556.0,34662.0,...,50.125,46.363636,47.272727,49.636364,44.5,13.818182,21.818182,35.363636,21.909091,20.363636


In [322]:
for feat in catergorical_features:
    s1[f'mode_{feat}_home_players'] = home_player_attribute[feat].mode()
s1

,id,match_api_id,home_team_api_id,date,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,...,avg_standing_tackle_home_players,avg_sliding_tackle_home_players,avg_gk_diving_home_players,avg_gk_handling_home_players,avg_gk_kicking_home_players,avg_gk_positioning_home_players,avg_gk_reflexes_home_players,mode_preferred_foot_home_players,mode_attacking_work_rate_home_players,mode_defensive_work_rate_home_players
0,19982,658907,8596,2010-03-13,43230.0,32553.0,35500.0,29049.0,32556.0,34662.0,...,49.636364,44.5,13.818182,21.818182,35.363636,21.909091,20.363636,right,medium,medium


In [ ]:
# feature engineering for player_attributes
# compute the mean for all players' overall_rating and potential
def get_player_attributes(match, player_attributes):
    '''
    for each player in a match, find the corresponding data about the player from the player_attributes set
    features: avg_overall_rating, min_overall_rating, max_overall_rating, avg_potential, min_overall_potential, max_overall_potential
    '''
    
    def _get_player_attributes(match, player_attributes, team='home'):
        # get players data for home_time
        home_player_cols = [f'home_player_{i}' for i in range(1,12)]
        away_player_cols = [f'away_player_{i}' for i in range(1,12)]
        # find all_ids of home_players
        
        

        for count, team in enumerate(team_api_ids):
            match_info = match[(match[teamid]==team)]
            team_attri_info = team_attributes[team_attributes['team_api_id']==team]
            match_dates = list(match_info['date'].values)
            for count_, date in enumerate(match_dates): 
                # get the team_attributes of the closest date
                team_attri_closest = team_attri_info[(team_attri_info['date']<= date)]
                team_attri_closest.sort_values(by='date', ascending=False, inplace=True)
                team_attri_closest = team_attri_closest.head(1)
                # drop unused cols
                team_attri_closest.drop(['id', 'team_fifa_api_id', 'date'], axis=1, inplace=True)
                # join team_attri_closest to the match_info set
                __match_joined = pd.merge(match_info[(match_info.date == date)], team_attri_closest, how='left', left_on=[teamid], right_on=['team_api_id'])
                # concat the joined sets by date
                if count_ == 0:
                    _match_joined = __match_joined
                else:
                    _match_joined = pd.concat([_match_joined, __match_joined], ignore_index=True)

            # concat the joined sets by team
            if count == 0:
                match_joined = _match_joined
            else:
                match_joined = pd.concat([match_joined, _match_joined], ignore_index=True)
#             print(f'count: {count}, no.matchs: {match_joined.shape}')
                
        return match_joined
    
    # get the team attributes for home_team and away_team separately
    match_home_team_attri = _get_team_attributes(match, team_attributes, teamid='home_team_api_id')
    match_away_team_attri = _get_team_attributes(match, team_attributes, teamid='away_team_api_id')
    
    # rename the features with prefix h_, aw to indicate feature for home_team, away_team
    feats = list(set(team_attributes.columns) - set(['id', 'team_fifa_api_id', 'team_api_id', 'date']))
    for feat in feats:
        match_home_team_attri.rename(columns={feat: f'h_{feat}'}, inplace=True)
        match_away_team_attri.rename(columns={feat: f'aw_{feat}'}, inplace=True)
    
    # join the 2 dataframes
    join_cols = ['id', 'country_id', 'league_id', 'season', 'stage','date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']
    match_home_team_attri = match_home_team_attri[join_cols+[f'h_{feat}' for feat in feats]]
    match_away_team_attri = match_away_team_attri[join_cols+[f'aw_{feat}' for feat in feats]]
    match_team_attri = pd.merge(match_home_team_attri, match_away_team_attri, how='inner', on=join_cols)
    
    # create compared features indicate the difference between home_team and away_team
    numerical_feats = team_attributes.drop(['id', 'team_fifa_api_id', 'team_api_id', 'date'], axis=1).select_dtypes(include='int64').columns.to_list()
    categorical_feats = team_attributes.select_dtypes(include='object').columns.to_list()
            
    for feat in feats:
        if feat in numerical_feats:
            match_team_attri[f'isHigher_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'h_{feat}'] > x[f'aw_{feat}'] else 0, axis=1)
        if feat in categorical_feats:
            match_team_attri[f'isDiff_{feat}'] = match_team_attri.apply(lambda x: 1 if x[f'h_{feat}'] != x[f'aw_{feat}'] else 0, axis=1)
    
#     # return only compared features 
#     compared_numerical_feats = [f'isHigher_{feat}' for feat in numerical_feats]
#     compared_categorical_feats = [f'isDiff_{feat}' for feat in categorical_feats]
#     match_info = ['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal','away_team_goal']
    
    return match_team_attri

### Historical Performance

Phân tích và giải thích các yếu tố sau có ảnh hưởng như thế nào đến kết quả trận đấu ?
    Profile đội bóng
    Profile cầu thủ
    Đội hình ra quân
    Lịch sử đối đầu giữa 2 đội

## Train/Test

In [ ]:
# Splitting the featured data into train and testing set

club_data = data[data['competitionid'] <> 4]
# Show the features latest game in competition id 4, which is the world cup.
data[data['competitionid'] == 4].iloc[0]

## Training the model

## Picking important features

## Testing